# Part 1 - Building a Language Model

In [1]:
import nltk
from nltk.collocations import *
from datetime import datetime
from collections import Counter
import os
import psutil

In [2]:
from sklearn.model_selection import train_test_split
import numpy as np

In [5]:
pid = os.getpid()
extraction_start = datetime.now()

file = open('Corpus\korpusmalti1.txt')
corpus = file.read()

tokens = nltk.word_tokenize(corpus)
RemoveSymbols(corpus)

py = psutil.Process(pid)
memoryUse = py.memory_info()[0]/2.**30
extraction_end = datetime.now()

extraction_time = dict()
extraction_time['extraction_time'] = extraction_end - extraction_start
print('Extraction Time(HH::MM:SS:ms) - {}\n\n'.format(extraction_time['extraction_time']))
print('Memory Use: ', memoryUse, 'GB')

Extraction Time(HH::MM:SS:ms) - 0:00:01.694533


Memory Use:  0.11978530883789062 GB


In [3]:
def RemoveSymbols(corpus):
    arr = []
    symbols = "“”‘’!\"#$€%&()*'+-,./:;<=>?@[\]^_`{|}~\n"
    
    for i in corpus:
        if i not in symbols:
            arr.append(i)
            
    return arr

In [6]:
bigrams = nltk.bigrams(tokens)
freqdist = nltk.FreqDist(bigrams)
for i,j in freqdist.items():
    print(i,j)

('<', 'text') 11
('text', 'id=') 11
('id=', "''") 1404
("''", 'academic1') 1
('academic1', "''") 1
("''", '>') 1404
('>', '<') 1730
('<', 'p') 316
('p', 'id=') 316
("''", '0') 22
('0', "''") 22
('<', 's') 1077
('s', 'id=') 1077
('>', 'L-') 117
('L-', 'DEF') 122
('DEF', 'il-') 3077
('il-', 'null') 3086
('null', 'gÄ§an') 35
('gÄ§an', 'NOUN') 35
('NOUN', 'gÄ§an') 35
('gÄ§an', 'null') 35
('null', 'prinÄ‹ipali') 9
('prinÄ‹ipali', 'ADJ') 10
('ADJ', 'prinÄ‹ipali') 10
('prinÄ‹ipali', 'null') 10
('null', "ta'") 703
("ta'", 'GEN') 836
('GEN', "ta'") 843
("ta'", 'null') 1935
('null', 'Conectando') 5
('Conectando', 'NOUN-PROP') 3
('NOUN-PROP', 'Conectando') 3
('Conectando', 'null') 5
('null', 'Mundos') 5
('Mundos', 'NOUN-PROP') 4
('NOUN-PROP', 'Mundos') 4
('Mundos', 'null') 5
('null', '(') 127
('(', 'X-PUN') 149
('X-PUN', '(') 149
('(', 'null') 149
('null', 'Malta') 13
('Malta', 'NOUN-PROP') 13
('NOUN-PROP', 'Malta') 13
('Malta', 'null') 13
('null', ')') 143
(')', 'X-PUN') 149
('X-PUN', ')') 149
(

('ambjentali', 'null') 28
('null', 'soÄ‹jali') 47
('soÄ‹jali', 'ADJ') 48
('ADJ', 'soÄ‹jali') 48
('soÄ‹jali', 'null') 48
('null', 'politiÄ‹i') 8
('politiÄ‹i', 'ADJ') 7
('ADJ', 'politiku') 8
('politiku', 'null') 10
('s-t-gÄ§', 'jwasslu') 1
('jwasslu', 'VERB') 2
('VERB', 'wassal') 26
('wassal', 'w-s-l') 26
('w-s-l', 'gÄ§all-') 3
('null', 'faqar') 14
('faqar', 'NOUN') 12
('NOUN', 'faqar') 12
('faqar', 'f-q-r') 14
('f-q-r', ',') 6
('null', 'inÄ¡ustizzja') 3
('inÄ¡ustizzja', 'NOUN') 3
('NOUN', 'inÄ¡ustizzja') 10
('inÄ¡ustizzja', 'null') 10
('null', 'kunflitt') 3
('kunflitt', 'NOUN') 4
('NOUN', 'kunflitt') 5
('kunflitt', 'null') 5
("''", '7') 22
('7', "''") 22
('>', 'Minbarra') 9
('Minbarra', 'PREP') 9
('PREP', 'Minbarra') 9
('Minbarra', 'null') 9
('null', 'dan') 93
('dan', 'PRON-DEM') 105
('null', 'is-') 71
('null', "''") 211
("''", 'X-PUN') 262
('X-PUN', "''") 262
("''", 'null') 262
('null', 'kien') 47
('kien', 'KIEN') 53
('null', 'wieÄ§ed') 24
('wieÄ§ed', 'NUM-WHD') 24
('NUM-WHD', 'wieÄ§ed

('taÄ§riÄ¡', 'NOUN') 9
('NOUN', 'Ä§ruÄ¡') 9
('Ä§ruÄ¡', 'Ä§-r-Ä¡') 12
('Ä§-r-Ä¡', 'introduttorju') 1
('introduttorju', 'ADJ') 1
('ADJ', 'introduttorju') 1
('introduttorju', 'null') 1
('null', 'informazzjoni') 13
('informazzjoni', 'NOUN') 18
('NOUN', 'informazzjoni') 18
('informazzjoni', 'null') 18
('null', 'materjal') 7
('materjal', 'NOUN') 9
('NOUN', 'materjal') 12
('materjal', 'null') 16
('null', 'promozzjonali') 1
('promozzjonali', 'ADJ') 1
('ADJ', 'promozzjonali') 1
('promozzjonali', 'null') 1
('null', 'r-') 30
('r-', 'DEF') 40
('null', 'gÄ§at-') 14
('gÄ§at-', 'PREP-DEF') 19
('gÄ§-l-m', 'li') 2
('null', 'pprovda') 1
('pprovda', 'VERB') 1
('VERB', 'pprovda') 1
('pprovda', 'null') 1
('null', 'gÄ§enuhom') 1
('gÄ§enuhom', 'VERB') 1
('null', 'jagÄ§mlu') 18
('jagÄ§mlu', 'VERB') 21
('gÄ§-m-l', 'l-') 11
('null', 'Ä§in') 15
('Ä§in', 'NOUN') 15
('NOUN', 'Ä§an') 15
('Ä§an', 'Ä§-j-n') 15
('Ä§-j-n', 'tagÄ§hom') 2
('null', 'extra') 1
('extra', 'ADJ') 1
('ADJ', 'extra') 1
('extra', 'null') 1
('nul

('jieqfu', 'VERB') 1
('VERB', 'waqaf') 5
('waqaf', 'w-q-f') 5
('w-q-f', ',') 3
('null', 'jaÄ§sbu') 4
('jaÄ§sbu', 'VERB') 4
('VERB', 'Ä§aseb') 11
('Ä§aseb', 'Ä§-s-b') 13
('Ä§-s-b', ',') 2
('null', 'jieÄ§du') 12
('jieÄ§du', 'VERB') 18
("'-Ä§-d", 'deÄ‹izjoni') 1
('deÄ‹izjoni', 'NOUN') 1
('NOUN', 'deÄ‹izjoni') 1
('deÄ‹izjoni', 'null') 1
('null', 'jaqsmu') 9
('jaqsmu', 'VERB') 9
('q-s-m', 'l-') 6
('null', 'opinjonijiet') 3
('opinjonijiet', 'NOUN') 3
('NOUN', 'opinjoni') 3
('opinjoni', 'null') 3
('null', 'tagÄ§hom') 81
('null', 'oÄ§rajn') 15
('oÄ§rajn', 'NOUN') 10
('NOUN', 'ieÄ§or') 11
('ieÄ§or', 'null') 31
('null', 'favur') 6
('favur', 'PREP') 6
('PREP', 'favur') 6
('favur', 'null') 6
('null', 'aÄ§jar') 14
('aÄ§jar', 'ADJ') 14
('ADJ', 'aÄ§jar') 14
('aÄ§jar', 'null') 16
("''", '26') 15
('26', "''") 15
('>', 'Mir-') 1
('Mir-', 'NOUN-PROP') 1
('NOUN-PROP', 'Mir-') 1
('Mir-', 'null') 1
('null', 'raÄ§al') 2
('raÄ§al', 'NOUN') 2
('NOUN', 'raÄ§al') 6
('raÄ§al', 'null') 6
('null', 'Å¼gÄ§ir') 3
('Å¼

('IÅ¼vilupp', 'null') 5
('null', 'Bniedem') 3
('Bniedem', 'NOUN') 3
('q-j-s', 'kemm') 2
('null', 'jieÄ§u') 4
('jieÄ§u', 'VERB') 4
("'-Ä§-d", 'l-') 5
('null', 'post') 11
('post', 'NOUN') 13
('NOUN', 'post') 19
('post', 'null') 19
('null', 'proÄ‹essi') 5
('proÄ‹essi', 'NOUN') 6
('null', 'Å¼vilupp') 11
('null', 'bosta') 10
('bosta', 'QUAN') 13
('QUAN', 'bosta') 13
('bosta', 'null') 13
('null', 'nazzjonijiet') 1
('nazzjonijiet', 'NOUN') 1
('NOUN', 'nazzjon') 1
('nazzjon', 'null') 1
('>', 'Ir-') 6
('Ir-', 'DEF') 6
('null', 'rapport') 3
('rapport', 'NOUN') 3
('NOUN', 'rapport') 3
('rapport', 'null') 3
("Ä¡-j-'", 'aÄ¡Ä¡ornat') 1
('aÄ¡Ä¡ornat', 'VERB') 1
('VERB', 'aÄ¡Ä¡ornat') 1
('aÄ¡Ä¡ornat', 'null') 2
('null', 'jinkludi') 1
('jinkludi', 'VERB') 1
('null', 'kwantitattiva') 1
('kwantitattiva', 'ADJ') 1
('ADJ', 'kwantitattiv') 1
('kwantitattiv', 'null') 1
('null', 'kwalitattiva') 1
('kwalitattiva', 'ADJ') 1
('ADJ', 'kwalitattiv') 1
('kwalitattiv', 'null') 1
('null', 'jinÄ¡abru') 3
('jinÄ¡abru',

...

('null', 'ekomija') 2
('ekomija', 'NOUN') 2
('null', 'Å¼vantaÄ¡Ä¡jata') 1
('Å¼vantaÄ¡Ä¡jata', 'PART-PASS') 1
('PART-PASS', 'Å¼vantaÄ¡Ä¡jat') 1
('Å¼vantaÄ¡Ä¡jat', 'null') 1
('f-q-r', 'ftit') 1
('null', 'negozjati') 2
('negozjati', 'NOUN') 2
('NOUN', 'negozjat') 2
('negozjat', 'null') 2
('f-h-m', "f'") 1
('null', 'GATS') 1
('GATS', 'X-ABV') 1
('X-ABV', 'GATS') 1
('GATS', 'null') 1
('null', 'Ä§eles') 1
('Ä§eles', 'VERB') 1
('Ä§-l-s', 'is-') 1
('s-Ä§-Ä§', 'saru') 1
('s-j-r', 'prodotti') 1
('null', 'taqlib') 1
('taqlib', 'NOUN') 1
('NOUN', 'taqlib') 1
('taqlib', 'q-l-b') 1
('q-l-b', 'tas-') 1
('null', 'kompetituri') 1
('kompetituri', 'NOUN') 1
('NOUN', 'kompetitur') 1
('kompetitur', 'null') 1
('b-r-j', "jista'") 1
('s-t-gÄ§', 'jbaxxi') 1
('jbaxxi', 'VERB') 1
('b-x-j', 'l-') 2
('null', 'tifrik') 1
('tifrik', 'NOUN') 1
('NOUN', 'tifrik') 1
('tifrik', 'f-r-k') 1
('f-r-k', 'tal-') 1
('null', 'konvenjenza') 1
('konvenjenza', 'NOUN') 1
('NOUN', 'konvenjenza') 1
('konvenjenza', 'null') 1
('null', 

('null', 'dgÄ§ajjef') 1
('dgÄ§ajjef', 'ADJ') 1
('ADJ', 'dgÄ§ajjef') 1
('dgÄ§ajjef', 'd-gÄ§-f') 1
('d-gÄ§-f', ',') 1
('null', 'kompliÄ‹i') 1
('kompliÄ‹i', 'ADJ') 1
('ADJ', 'kompliÄ‹i') 1
('kompliÄ‹i', 'null') 1
('>', 'JistgÄ§u') 1
('JistgÄ§u', 'VERB') 1
('s-t-gÄ§', 'il-') 1
('>', "Tista'") 1
("Tista'", 'VERB') 1
('s-t-gÄ§', 'azjenda') 1
('null', 'teÅ¼isti') 2
('teÅ¼isti', 'VERB') 2
('>', 'Le') 1
('Le', 'INT') 1
('INT', 'Le') 1
('Le', 'null') 1
('>', 'Lanqas') 1
('Lanqas', 'FOC') 1
('FOC', 'Lanqas') 1
('Lanqas', 'null') 1
('s-t-gÄ§', 'jwelled') 1
('jwelled', 'VERB') 1
('VERB', 'welled') 1
('welled', 'w-l-d') 1
('w-l-d', 'azjenda') 1
('s-t-gÄ§', 'jiggarantixxi') 1
('jiggarantixxi', 'VERB') 1
('null', 'limitati') 1
('limitati', 'PART-PASS') 1
('s-t-gÄ§', 'jiffirma') 1
('jiffirma', 'VERB') 1
('f-h-m', 'kummerÄ‹jali') 1
('null', 'jillimitaw') 1
('jillimitaw', 'VERB') 1
('null', 'jÅ¼omm') 1
('Å¼-m-m', 'taÄ§t') 1
('t-Ä§-t', 'kontroll') 1
('s-j-r', 'qawwija') 1
('q-w-j', 'l-') 1
('n-Ä§-j', 'tar

('trejdunjins', 'NOUN') 1
('attivi', 'NOUN') 1
('NOUN', 'attiv') 1
('null', 'Ä¡imagÄ§tejn') 1
('Ä¡imagÄ§tejn', 'NOUN') 1
('Ä¡-m-gÄ§', 'Å¼orna') 1
('Å¼-w-r', 'diversi') 1
('null', 'intervistajna') 1
('intervistajna', 'VERB') 1
('VERB', 'intervista') 1
('gÄ§-t-j', 'ftit') 1
('Ä§-j-l', 'tas-') 1
('null', 'Ä§arxa') 1
('Ä§arxa', 'NOUN') 1
('NOUN', 'aÄ§rax') 1
('aÄ§rax', 'Ä§-r-x') 1
('Ä§-r-x', 'li') 1
('Ä§-d-m', 'fiha') 2
('x-r-j', 'aÄ§na') 1
('null', 'magÄ§Å¼ul') 1
('magÄ§Å¼ul', 'PART-PASS') 1
('gÄ§-Å¼-l', 'gÄ§all-') 1
('Isvezja', 'NOUN-PROP') 4
('NOUN-PROP', 'Isvezja') 4
('Isvezja', 'null') 4
("'-Ä§-d", 'qatgÄ§a') 1
('qatgÄ§a', 'NOUN') 1
('NOUN', 'qatgÄ§a') 1
('qatgÄ§a', 'q-t-gÄ§') 1
('q-t-gÄ§', ':') 1
('null', 'qalziet') 1
('null', 'mibjugÄ§') 1
('mibjugÄ§', 'PART-PASS') 1
('PART-PASS', 'mibjugÄ§') 2
('mibjugÄ§', 'b-j-gÄ§') 2
('b-j-gÄ§', "f'") 1
('s-w-j', 'iktar') 1
('null', 'jirnexxilek') 1
('jirnexxilek', 'VERB') 1
('gÄ§-j-x', 'sew') 1
('s-t-gÄ§', 'jagÄ§mlu') 2
('null', 'Clean') 3
('Cle

('PART-PASS', 'gÄ§abba') 1
('gÄ§abba', 'gÄ§-b-j') 1
('gÄ§-b-j', 'bil-') 1
('b-d-j', 'attivitÃ') 1
('null', 'inqajmu') 1
('inqajmu', 'VERB') 1
('q-j-m', 'kuxxjenza') 1
('kuxxjenza', 'NOUN') 1
('NOUN', 'kuxxjenza') 1
('kuxxjenza', 'null') 1
('null', 'Afrikan') 1
('Afrikan', 'NOUN-PROP') 1
('NOUN-PROP', 'Afrikan') 1
('Afrikan', 'null') 1
('b-d-j', 'niÄ¡bru') 1
('niÄ¡bru', 'VERB') 1
('null', 'edukattiv') 1
('edukattiv', 'ADJ') 1
('null', 'sanitarju') 1
('sanitarju', 'NOUN') 1
('NOUN', 'sanitarju') 1
('sanitarju', 'null') 1
('null', 'spiÄ‹Ä‹at') 1
('spiÄ‹Ä‹at', 'VERB') 1
('null', 'telqet') 1
('telqet', 'VERB') 1
('t-l-q', 'mit-') 1
('Angola', 'NOUN') 4
('NOUN', 'angolu') 4
('angolu', 'null') 4
('null', 'Ä§dimna') 1
('Ä§dimna', 'VERB') 1
('null', 'Ministeru') 1
('Ministeru', 'NOUN') 1
('NOUN', 'Ministeru') 1
('Ministeru', 'null') 1
('null', 'Affarijiet') 1
('Affarijiet', 'NOUN') 1
('gÄ§-j-n', 'lir-') 1
('null', 'veterani') 1
('veterani', 'ADJ') 1
('ADJ', 'veteran') 1
('veteran', 'null') 1
('

('k-l-l', 'tkun') 1
('>', 'IÅ¼jed') 1
('IÅ¼jed', 'ADV') 1
('iÅ¼jed', 'Å¼-j-d') 1
('Å¼-j-d', 'ma') 1
('null', 'qorbot') 1
('qorbot', 'VERB') 1
('VERB', 'qorob') 1
('qorob', 'q-r-b') 1
('q-r-b', 'id-') 1
('null', 'data') 1
('data', 'NOUN') 1
('NOUN', 'data') 1
('data', 'null') 1
('t-l-q', ',') 1
('null', 'bdejt') 1
('bdejt', 'VERB') 1
('b-d-j', 'inÄ§ossni') 1
('inÄ§ossni', 'VERB') 1
('Ä§-s-s', 'eÄ‹itata') 1
('eÄ‹itata', 'PART-PASS') 1
('PART-PASS', 'eÄ‹itat') 1
('eÄ‹itat', 'null') 1
('null', 'beÅ¼gÄ§ana') 1
('beÅ¼gÄ§ana', 'NOUN') 1
('NOUN', 'beÅ¼gÄ§an') 1
('beÅ¼gÄ§an', 'b-Å¼-gÄ§') 1
('b-Å¼-gÄ§', 'wkoll') 1
('>', 'Kelli') 1
('Kelli', 'VERB-PSEU') 1
('k-l-l', 'rasi') 1
('r-j-s', 'ddur') 1
('ddur', 'VERB') 2
('d-w-r', 'bil-') 1
('null', 'tekniku') 1
('>', 'Tlaqna') 1
('Tlaqna', 'VERB') 1
('t-l-q', 'minn') 1
('null', '16') 1
('16', 'X-DIG') 1
('X-DIG', '16') 1
('16', 'null') 1
('Ä§-j-n', 'lokali') 1
('h-m-m', 'konna') 1
('null', 'Helsingor') 1
('Helsingor', 'NOUN-PROP') 1
('NOUN-PROP', 'Hels

In [6]:
def Split(corpus):
    
    corpus = open(corpus, "r")
    
    for line in corpus:
        
        words = np.array(line.split())
        
    train, test = train_test_split(words, shuffle = False)
    
    return train, test

# Part 2 - Building a Language Model

## Vanilla

In [7]:
def VanillaUnigram(train):
    
    model = Counter(train)
    
    for word in model:
        model[word] = model[word]/len(train)
        
    return model

def VanillaBigram(train):
    
    model = Counter([(word, train[i + 1]) for i, word in enumerate(train[:-1])])
    counter = Counter(train)
    
    for word in model:
        model[word] = model[word]/counter[word[0]]
        
    return model

def VanillaTrigram(train):
    
    bigram = Counter([(word, train[i + 1]) for i, word in enumerate(train[:-1])])
    trigram = Counter([(word, train[i + 1], train[i + 2]) for i, word in enumerate(train[:-2])])
    
    for word in trigram:
        trigram[word] = trigram[word]/bigram[(word[0], word[1])]
        
    return trigram

## Laplace 

In [8]:
def LaplaceUnigram(train):
    
    model = Counter(train)
    
    for word in model:
        model[word] = (model[word]+1)/len(train)
        
    return model

## UNK

In [9]:
def UNKUnigram(train):
    
    counter = Counter(train)
    model = {}
    model["<UNK>"] = 0
    
    for word in counter:
        if counter[word] == 1:
            model["<UNK>"] += 1
            
        else:
            model[word] = counter[word]
            
    for word in model:
        model[word] = model[word]/len(train)
        
    return model

def UNKBigram(train):
    
    unigram = UNKUnigram(train)
    
    for i, word in enumerate(train):
        if not (word in unigram):
            train[i] = "<UNK>"
            
    return VanillaBigram(train)

def UNKTrigram(train):
    
    unigram = UNKUnigram(train)
    
    for i, word in enumerate(train):
        if not (word in unigram):
            train[i] = "<UNK>"
            
    return VanillaTrigram(train)

## Probability

In [10]:
def UnigramProbability(unigram, sentence, word):
    return unigram[word]

def BigramProbability(bigram, sentence, word):
    if (sentence[-1], word) in bigram:
        return bigram[sentence[-1],word]
    
    else:
        return 0
    
def TrigramProbability(trigram, sentence, word): 
    if (sentence[-2],sentence[-1], word) in trigram:
        return trigram[sentence[-2],sentence[-1],word]
    
    else:
        return 0

## Generate

In [2]:
def Generate(train, bigram, sentence, last = "", count = None):
    
    print("Via What Model would you like to Generate?\n")
    choice = input("Enter Model:")
     
    print("Generating via " + choice + "...")
    
    if choice is "Laplace":
        if (count != 0 and (sentence[-1] != last)):
            
            counter = Counter(train)
            bigrams = {}
            
            for word in counter:
                x = (sentence[-1], word)
                if x in bigram.keys():
                    bigram[sentence[-1],word] = (bigram[sentence[-1], word] +1)/(counter[sentence[-1]] + len(counter))
                    
                else:
                    bigrams[sentence[-1], word] = 1/(counter[sentence[-1]] + len(counter))
                    
            weights = np.array(list(bigram.values()))
            norm_weights = weights/np.sum(weights)
            
            resample = np.random.multinomial(1, norm_weights)
            key = list(resample).index(1)
            value = list(bigram.keys())[key]
            
            sentence.append(value[1])
            
            if count != None:
                Generate(train, bigram, sentence, last, count -1)
                
            else:
                Generate(train, bigram, sentence, last)
                
        return sentence
        
    elif choice is "UNK":
        
        
    elif choice is "Vanilla":
        
        
    else:
        print("Invalid Choice\n")

<>:8: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:39: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:8: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:39: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-2-242b316673f4>:8: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if choice is "Laplace":
<ipython-input-2-242b316673f4>:39: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif choice is "UNK":


SyntaxError: 'break' outside loop (<ipython-input-2-242b316673f4>, line 41)